In [1]:
import torch
from einops import rearrange

# Create a 4D tensor of shape (batch, channels, height, width)
tensor = torch.rand(10, 3, 32, 32)  # Example: a batch of 10 RGB images 32x32
# Rearrange to (batch, height, width, channels) for image processing libraries that expect this format
rearranged = rearrange(tensor, 'b c h w -> b h w c')

c:\Users\pitiw\miniconda3\envs\open-oasis\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Reduce the tensor's channel dimension by taking the mean, resulting in a grayscale image \
Since the "c" is vanished so it does mean there's agrregation there. \
It averages out the channel column. So, if there're red green blue columns, it averages all three.

In [ ]:
from einops import reduce

# Reduce methods 'min', 'max', 'sum', 'mean', 'prod', 'any', 'all'
grayscale = reduce(tensor, 'b c h w -> b h w', 'mean')

In [5]:
from einops import repeat

# Repeat each image in the batch 4 times along a new dimension
# It duplicated the batch dimension
repeated = repeat(tensor, 'b c h w -> (repeat b) c h w', repeat=4)

Spliting and Merging

In [ ]:
# Act like (c rgb) a singlw column but later want to separate c away as a separate vec dimension
# Not just act only but we can separate them
print(tensor.shape)
x = rearrange(tensor, 'b (c rgb) h w -> rgb b c h w', rgb=3)
print(x.shape)

torch.Size([10, 3, 32, 32])
torch.Size([3, 10, 1, 32, 32])


In [11]:
# Split channels
red, green, blue = rearrange(tensor, 'b (c rgb) h w -> rgb b c h w', rgb=3)

# Example processing (identity here)
processed_red, processed_green, processed_blue = red, green, blue
# Merge channels back
merged = rearrange([processed_red, processed_green, processed_blue], 'rgb b c h w -> b (rgb c) h w')

In [2]:
# Flatten spatial dimensions
flattened = rearrange(tensor, 'b c h w -> b (c h w)')

# Example neural network operation
# output = model(flattened)
# Unflatten back to spatial dimensions (assuming output has shape b, features)
# unflattened = rearrange(output, 'b (c h w) -> b c h w', c=3, h=32, w=32)

In [3]:
flattened.shape

torch.Size([10, 3072])

## Attention without eniops

In [4]:
import torch
import torch.nn.functional as F
from einops import rearrange

def simplified_self_attention(q, k, v):
    """
    A simplified self-attention mechanism.
    Args:
        q, k, v (torch.Tensor): Queries, Keys, and Values. Shape: [batch_size, num_tokens, feature_dim]
    Returns:
        torch.Tensor: The result of the attention mechanism.
    """
    # Compute the dot product between queries and keys
    scores = torch.matmul(q, k.transpose(-2, -1))
    
    # Apply softmax to get probabilities
    attn_weights = F.softmax(scores, dim=-1)
    
    # Multiply by values
    output = torch.matmul(attn_weights, v)
    return output

# Example tensors representing queries, keys, and values
batch_size, num_tokens, feature_dim = 10, 16, 64
q = torch.rand(batch_size, num_tokens, feature_dim)
k = torch.rand(batch_size, num_tokens, feature_dim)
v = torch.rand(batch_size, num_tokens, feature_dim)
# Apply self-attention
attention_output = simplified_self_attention(q, k, v)
print("Output shape:", attention_output.shape)

Output shape: torch.Size([10, 16, 64])


In [ ]:
[x for x in (q, k, v)]

In [ ]:
def multi_head_self_attention(q, k, v, num_heads=8):
    """
    Multi-head self-attention using Einops for splitting and merging heads.
    """
    batch_size, num_tokens, feature_dim = q.shape
    head_dim = feature_dim // num_heads
    
    # Split into multiple heads
    q, k, v = [
        rearrange(x, 'b t (h d) -> b h t d', h=num_heads)
        for x in (q, k, v)
    ]
    
    # Apply self-attention to each head
    output = simplified_self_attention(q, k, v)
    
    # Merge the heads back
    output = rearrange(output, 'b h t d -> b t (h d)')
    return output

# Apply multi-head self-attention
multi_head_attention_output = multi_head_self_attention(q, k, v)
print("Multi-head output shape:", multi_head_attention_output.shape)
